## Assignment 3 


Energy indicator xls sheet 

In [2]:
import pandas as pd
import numpy as np
import re

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [3]:
def convert(item):
    if item["Energy Supply"] == "...":
        return np.NaN
    else:
        item["Energy Supply"] *= 10**6
        return item["Energy Supply"]

def e_rename(item):
    country = item
    if country == "Republic of Korea":
        return "South Korea"
    elif "United States of America" in country:
        return "United States"
    elif "United Kingdom of Great Britain and Northern Ireland" in country:
        return "United Kingdom"
    elif "China, Hong Kong Special Administrative Region" in country:
        return "Hong Kong"
    else:
        return country

def fmt(item):
    search = re.compile(r"\(.+?\)|[0-9]")
    if search.findall(item):
        for i in search.findall(item):
            item = item.replace(i,'')
        if item[-1] == " ":
            item = item[:len(item)-1]
    return item
    
def energ():
    energy = pd.read_excel('Energy Indicators.xls',header=17,skipfooter=38)
    energy = energy.drop(['Unnamed: 0','Unnamed: 1'],axis=1)
    energy = energy.rename(columns={'Unnamed: 2':'Country',
                                    'Petajoules':'Energy Supply', 
                                    'Gigajoules':'Energy Supply per Capita', 
                                    '%':'% Renewable'})
    energy['Energy Supply'] = energy.apply(convert,axis=1)
    energy['Country'] = list(map(e_rename,energy['Country']))
    energy['Country'] = list(map(fmt,energy['Country']))
    return energy

def g_rename(item):
    if "Korea, Rep." in item:
        return "South Korea"
    elif "Iran, Islamic Rep." in item:
        return "Iran"
    elif "Hong Kong SAR, China" in item:
        return "Hong Kong"
    else:
        return item

def gdp():
    GDP = pd.read_csv('world_bank.csv',header=4)
    GDP['Country Name'] = list(map(g_rename,GDP['Country Name'])) 
    return GDP

def scimen():
    ScimEn = pd.read_excel('scimagojr-3.xlsx')
    return ScimEn

def answer_one():
    energy = energ()
    GDP = gdp()
    ScimEn = scimen()
    temp = pd.merge(GDP,energy,how='inner',left_on='Country Name',right_on='Country')
    temp = temp.drop('Country',axis=1)
    #print(temp)
    DF = pd.merge(temp,ScimEn,how='inner',left_on='Country Name',right_on='Country')
    DF = DF.drop('Country',axis=1)
    DF.reset_index()
    DF = DF.set_index(['Country Name'])
    DF = DF.loc[:,['Rank', 'Documents', 'Citable documents','Citations', 'Self-citations', 'Citations per document',
                   'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable',
                   '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    DF = DF.sort_values(by='Rank')[:15]
    return DF
    
answer_one()



,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country Name,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [7]:
102

102

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [3]:
def avg(row):
    data = row[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    return np.mean(data)

def answer_three():
    Top15 = answer_one()
    df3 = Top15.copy()
    avg1 = df3.apply(avg,axis=1)
    avgGDP = pd.Series(avg1).sort_values(ascending=True)
    return avgGDP

answer_three()

Country Name
Iran                  4.441558e+11
South Korea           1.106715e+12
Australia             1.164043e+12
Spain                 1.418078e+12
Russian Federation    1.565459e+12
Canada                1.660647e+12
India                 1.769297e+12
Italy                 2.120175e+12
Brazil                2.189794e+12
United Kingdom        2.487907e+12
France                2.681725e+12
Germany               3.493025e+12
Japan                 5.542208e+12
China                 6.348609e+12
United States         1.536434e+13
dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [5]:
def avg(row):
    data = row[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    return np.mean(data)

def chng_gdp(row):
    return abs(row['2015']-row['2006'])

def answer_four():
    Top15 = answer_one()
    df4 = Top15.copy()
    avg1 = df4.apply(avg,axis=1)
    df4['avg'] = avg1
    df4 = df4.sort_values(by='avg',ascending=False)
    chngGDP = chng_gdp(df4.iloc[5][['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']])
    return chngGDP
    
answer_four()

246702696075.3999

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [37]:
def mean_energy(row):
    return np.mean(row)

def answer_five():
    Top15 = answer_one()
    df5 = Top15.copy()
    mean_energy_cap = mean_energy(df5['Energy Supply per Capita'])
    return mean_energy_cap

answer_five()

157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [40]:
def answer_six():
    Top15 = answer_one()
    df6 = Top15.copy()
    df6 = df6.sort_values(by='% Renewable',ascending=False)
    country, value = df6.index[0], df6['% Renewable'][0]
    return (country, value)

answer_six()

('Brazil', 69.64803)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [45]:
def ratio(row):
    return np.divide(row['Self-citations'],row['Citations'])

def answer_seven():
    Top15 = answer_one()
    df7 = Top15.copy()
    df7['ratio_citat'] = df7.apply(ratio,axis=1)
    df7 = df7.sort_values(by='ratio_citat',ascending=False)
    country, value = df7.index[0], df7['ratio_citat'][0]
    return (country, value)

answer_seven()


('China', 0.6893126179389422)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [49]:
def calc_pop(row):
    return np.divide(row['Energy Supply'],row['Energy Supply per Capita'])

def answer_eight():
    Top15 = answer_one()
    df8 = Top15.copy()
    df8['est_pop'] = df8.apply(calc_pop, axis=1)
    df8 = df8.sort_values(by='est_pop',ascending=False)
    return df8.index[2]

answer_eight()

'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [51]:
def answer_nine():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    correlation = np.correlate(Top15['Citable docs per Capita'],Top15['Energy Supply per Capita'])
    return correlation[0]

answer_nine()

0.6091199998548664

In [ ]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [6]:
def answer_ten():
    Top15 = answer_one()
    med_renew = np.median(Top15['% Renewable'])
    category=[]
    for val in Top15['% Renewable']:
        if val >= med_renew:
            category.append(1)
        else:
            category.append(0)
    Top15['category'] = category
    Top15 = Top15.sort_values(by='Rank')
    HighRenew = Top15['category']
    return HighRenew

answer_ten()

Country Name
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: category, dtype: int64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [5]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

def answer_eleven():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    continents = [ContinentDict[i] for i in list(Top15.index )]
    Top15['continents'] = continents
    Top15['PopEst'] = pd.to_numeric(Top15['PopEst'])
    df = Top15.groupby('continents')['PopEst'].agg({'size':'count','sum':'sum','mean':'mean','std':'std'})
    return df



### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [9]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}


def answer_twelve():
    Top15 = answer_one()
    Top15['renew_gp'] = pd.cut(Top15['% Renewable'],5)
    Top15['continents'] = [ContinentDict[i] for i in list(Top15.index )]
    df = Top15.groupby(['continents','renew_gp'])['Rank'].count()
    return df
    
answer_twelve()

continents     renew_gp        
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Australia      (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    0
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
               (42.701, 56.174]    0
               (56.174, 69.648]    0
North America  (2.212, 15.753]     1
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
South America  (2.212, 15.753]     0
               (15.753, 29.227]    0
               (29.227, 42.701]    0
               (42.701, 56.174]    0
               (56.174, 69.648]    1
Name: Rank, dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*


In [4]:
def formatw(val):
    temp = str(val)
    temp = temp.split('.')
    num = temp[0]
    ptr = len(num)-1
    count=0
    num_word = ""
    while ptr >= 0:
        if count == 3:
            num_word = num_word + ','
            count = 0
        num_word = num_word + num[ptr]
        ptr -= 1
        count += 1
    return num_word[::-1] + '.' + temp[1]

def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    values = []
    for val in Top15['PopEst']:
        values.append(formatw(val))
    PopEst = pd.Series(values,index=Top15.index)
    return PopEst

answer_thirteen()

Country Name
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
dtype: object